In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
def cv_show(window_name, img_name):
    cv2.imshow(window_name, img_name)
    cv2.waitKey(5000)
    cv2.destroyAllWindows()

## 读取视频

In [5]:
video = cv2.VideoCapture('video.mp4')

# 循环读取视频
while True:
    ret, frame = video.read()
    if ret == True:
        cv2.imshow('video', frame)
        
    key = cv2.waitKey(1)
    # 用户按esc退出
    if key ==27:
        break
        
# 释放资源
video.release()
cv2.destroyAllWindows()

## 去除背景实例

In [20]:
cap = cv2.VideoCapture('video.mp4')
# 创建mog对象
mog = cv2.bgsegm.createBackgroundSubtractorMOG()

while True:
    ret, frame = cap.read()
    if ret ==True:
        fgmask = mog.apply(frame)
        cv2.imshow('video', fgmask)
    key = cv2.waitKey(1)
    # 用户按esc退出
    if key ==27:
        break
        
# 释放资源
video.release()
cv2.destroyAllWindows()  
    

## 形态学识别车辆

In [17]:
video = cv2.VideoCapture('video.mp4')

mog = cv2.bgsegm.createBackgroundSubtractorMOG()
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

w_min = 90
h_min = 82

line_high = 600
offset = 9  # 偏移量

cars = []
count = 0

# 计算外接矩形中心点
def center(x, y, w, h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = int(x) + x1
    cy = int(y) + y1
    return cx, cy
# 循环读取视频
while True:
    ret, frame = video.read()
    if ret == True:
        # 转换为灰度图
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # 高斯去噪
        gaus_blur = cv2.GaussianBlur(gray, (3, 3), 5)
        # 中值去噪
        medi_blur = cv2.medianBlur(gray, 5)
        mask = mog.apply(gaus_blur)
        # 腐蚀
        erode = cv2.erode(mask, kernel)
        # 膨胀
        dilate = cv2.dilate(erode, kernel, iterations=2)
        # 闭运算（消除内部像素小块）
        close = cv2.morphologyEx(dilate, cv2.MORPH_CLOSE, kernel)

        # 查找轮廓
        contours, h = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # 画出检测线
        cv2.line(frame, (10, line_high), (1200, line_high), (255, 255, 0), 3)
        # 画出所有轮廓
        for contour in contours:
            # 画外接矩形
            (x, y, w, h) = cv2.boundingRect(contour)
            # 根据w, h 过滤掉不需要的矩形
            is_valid = (w >= w_min) &  (h >= h_min)
            if not is_valid:
                continue
            # 到这的矩形， 就会被画出， 要求坐标点为整数
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            # 把车抽象为一个点，即外接矩形的中心点
            cpoint = center(x, y, w, h)
            cars.append(cpoint)
            cv2.circle(frame, (cpoint), 5, (0, 0, 255), -1)
            
            # 判断汽车是否过线
            for (x, y) in cars:
                if y > (line_high-offset) and y < (line_high+offset):
                    # 落入统计线区间，开始计数
                    count += 1
                    cars.remove((x, y))
        # 显示视频
        cv2.putText(frame, 'Vehicle Count:' + str(count), (500, 60), cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 255), 5 )
        cv2.imshow('frame', frame)
        
    key = cv2.waitKey(1)
    # 用户按esc退出
    if key ==27:
        break
        
# 释放资源
video.release()
cv2.destroyAllWindows()